In [1]:
import import_ipynb
import os
import pandas as pd
from WNDomains import WordNetDomains 
wnd = WordNetDomains(wordnet_home='.')

importing Jupyter notebook from WNDomains.ipynb


In [2]:
path= '../trame'

#domains_trama = set()  


list1=[1990, 2000, 2010]
dict_all_domains={}

for dirpath, dirnames, files in os.walk(path):
    print(f'Found directory: {dirpath}')

    dict_domains = {}
    for file in files:
        if file[0] == '.':
            continue

        domains_trama_all = set()

        with open(dirpath+ "/" + file, 'r', encoding='Latin-1') as f:
            text=f.read()
            words = text.split()

            file=file.split('.')[0]
            decade=dirpath.split('/')[-1]

            for word in words:
               # domains_trama|=(wnd.get_domains(word, 'n', first_sense_only=True))   ---- da usare per solo i nomi
                domains_trama_all |= (wnd.get_domains(word, 'n', first_sense_only=True))
                #domains_trama.discard('set()')



            for domain in domains_trama_all:
                if domain in dict_domains:
                    dict_domains[domain].append(file)
                else:
                    dict_domains[domain] = [file]

    for d in dict_domains:
        if decade in list1:
            if len(dict_domains[d]) >= (len(files)/2):
                if d in dict_all_domains:
                    dict_all_domains[d].append(decade)
                else:
                    dict_all_domains[d] = [decade]
        else:
            if len(dict_domains[d]) >= (len(files)*(0.75)):
                if d in dict_all_domains:
                    dict_all_domains[d].append(decade)
                else:
                    dict_all_domains[d] = [decade]
                                    

Found directory: ../trame
Found directory: ../trame/1990
Found directory: ../trame/1940
Found directory: ../trame/1970
Found directory: ../trame/1980
Found directory: ../trame/1930
Found directory: ../trame/1960
Found directory: ../trame/1950
Found directory: ../trame/2000
Found directory: ../trame/2010


In [6]:
full_df = pd.DataFrame(columns=['1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010'])

for dom, decs in dict_all_domains.items():
    full_df.loc[dom] = ['Y' if (str(dec) in decs) else 'N' for dec in range(1930, 2010 +1, 10)]
    
full_df.sort_index(inplace=True)

In [7]:
full_df.head()

,1930,1940,1950,1960,1970,1980,1990,2000,2010
administration,Y,N,Y,Y,Y,Y,Y,Y,Y
anatomy,Y,N,Y,Y,Y,Y,Y,Y,Y
animals,Y,N,Y,Y,Y,Y,Y,Y,Y
anthropology,Y,N,N,N,N,N,N,N,N
art,N,N,N,N,N,Y,N,N,N


In [190]:
# DA NON FARE ANDARE
#tentativo inutile e cmq non riuscito cmq di ordinare per anno
df=df.fillna(0)
columns=list(df.columns)

for column in columns:
    year = 1930
    for j in range (0, 9):
        if df.iloc[j, df.columns.get_loc(column)] == 0:
            year+=1
            continue
        df.iloc[j, df.columns.get_loc(column)] = year
        year += 10

In [196]:
df_null = df.isnull().sum(axis = 0)
df_null=df_null.to_frame().reset_index()

In [197]:
df_null.rename(columns={'index': 'domain', 0:'counts'}, inplace=True)
df_null

,domain,counts
0,tourism,0
1,plants,0
2,town_planning,0
3,economy,0
4,military,0
...,...,...
91,golf,8
92,philosophy,8
93,hydraulics,8
94,card,8


In [198]:
# se uno vuol fare dei check sui valori nulli (ossia )
for g in range (0,95):
    if df_null['counts'][g] > 5:
        print(df_null['domain'][g])

agriculture
acoustics
betting
hunting
sexuality
electricity
grammar
computer_science
football
pedagogy
occultism
painting
swimming
vehicles
post
telephony
enterprise
telegraphy
geometry
atomic_physic
earth
optics
golf
philosophy
hydraulics
card


In [199]:
df.to_csv(r"/Users/valeriaguttilla/Desktop/MASTER/PROGETTONE/ANALISI_TESTUALE/ANALISI TRAME/TRAME/ANIMATED/csv_trame.csv")